# Process S3 Parquet Files - Scratch Environment
2023.01.30

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.types import StructType, StructField, StringType, LongType, DateType, DoubleType
from pyspark.sql.functions import col, lit, array, explode, arrays_zip, posexplode, \
    sum as sum_, max as max_, min as min_, year, month

In [2]:
import os
os.environ["AWS_PROFILE"] = "service_wp"

In [3]:
!echo $AWS_PROFILE

service_wp


In [4]:
# set spark configuration
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('process-weather-stations') \
    .set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.2') \
    .set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.profile.ProfileCredentialsProvider')

In [5]:
# instantiate Spark session
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

23/02/01 15:37:49 WARN Utils: Your hostname, hp-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.167 instead (on interface wlp1s0)
23/02/01 15:37:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/kishan/spark/spark-3.3.1-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kishan/.ivy2/cache
The jars for the packages stored in: /home/kishan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-535359e6-7bd5-4b4b-af19-5931e81e89db;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 353ms :: artifacts dl 14ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------------

23/02/01 15:37:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
station_schema = StructType([StructField('STATION', StringType(), True),
 StructField('NAME', StringType(), True),
 StructField('LATITUDE', DoubleType(), True),
 StructField('LONGITUDE', DoubleType(), True),
 StructField('ELEVATION', DoubleType(), True),
 StructField('DATE', DateType(), True),
 StructField('HR00Val', LongType(), True),
 StructField('HR00MF', StringType(), True),
 StructField('HR00QF', StringType(), True),
 StructField('HR00S1', StringType(), True),
 StructField('HR00S2', StringType(), True),
 StructField('HR01Val', LongType(), True),
 StructField('HR01MF', StringType(), True),
 StructField('HR01QF', StringType(), True),
 StructField('HR01S1', StringType(), True),
 StructField('HR01S2', StringType(), True),
 StructField('HR02Val', LongType(), True),
 StructField('HR02MF', StringType(), True),
 StructField('HR02QF', StringType(), True),
 StructField('HR02S1', StringType(), True),
 StructField('HR02S2', StringType(), True),
 StructField('HR03Val', LongType(), True),
 StructField('HR03MF', StringType(), True),
 StructField('HR03QF', StringType(), True),
 StructField('HR03S1', StringType(), True),
 StructField('HR03S2', StringType(), True),
 StructField('HR04Val', LongType(), True),
 StructField('HR04MF', StringType(), True),
 StructField('HR04QF', StringType(), True),
 StructField('HR04S1', StringType(), True),
 StructField('HR04S2', StringType(), True),
 StructField('HR05Val', LongType(), True),
 StructField('HR05MF', StringType(), True),
 StructField('HR05QF', StringType(), True),
 StructField('HR05S1', StringType(), True),
 StructField('HR05S2', StringType(), True),
 StructField('HR06Val', LongType(), True),
 StructField('HR06MF', StringType(), True),
 StructField('HR06QF', StringType(), True),
 StructField('HR06S1', StringType(), True),
 StructField('HR06S2', StringType(), True),
 StructField('HR07Val', LongType(), True),
 StructField('HR07MF', StringType(), True),
 StructField('HR07QF', StringType(), True),
 StructField('HR07S1', StringType(), True),
 StructField('HR07S2', StringType(), True),
 StructField('HR08Val', LongType(), True),
 StructField('HR08MF', StringType(), True),
 StructField('HR08QF', StringType(), True),
 StructField('HR08S1', StringType(), True),
 StructField('HR08S2', StringType(), True),
 StructField('HR09Val', LongType(), True),
 StructField('HR09MF', StringType(), True),
 StructField('HR09QF', StringType(), True),
 StructField('HR09S1', StringType(), True),
 StructField('HR09S2', StringType(), True),
 StructField('HR10Val', LongType(), True),
 StructField('HR10MF', StringType(), True),
 StructField('HR10QF', StringType(), True),
 StructField('HR10S1', StringType(), True),
 StructField('HR10S2', StringType(), True),
 StructField('HR11Val', LongType(), True),
 StructField('HR11MF', StringType(), True),
 StructField('HR11QF', StringType(), True),
 StructField('HR11S1', StringType(), True),
 StructField('HR11S2', StringType(), True),
 StructField('HR12Val', LongType(), True),
 StructField('HR12MF', StringType(), True),
 StructField('HR12QF', StringType(), True),
 StructField('HR12S1', StringType(), True),
 StructField('HR12S2', StringType(), True),
 StructField('HR13Val', LongType(), True),
 StructField('HR13MF', StringType(), True),
 StructField('HR13QF', StringType(), True),
 StructField('HR13S1', StringType(), True),
 StructField('HR13S2', StringType(), True),
 StructField('HR14Val', LongType(), True),
 StructField('HR14MF', StringType(), True),
 StructField('HR14QF', StringType(), True),
 StructField('HR14S1', StringType(), True),
 StructField('HR14S2', StringType(), True),
 StructField('HR15Val', LongType(), True),
 StructField('HR15MF', StringType(), True),
 StructField('HR15QF', StringType(), True),
 StructField('HR15S1', StringType(), True),
 StructField('HR15S2', StringType(), True),
 StructField('HR16Val', LongType(), True),
 StructField('HR16MF', StringType(), True),
 StructField('HR16QF', StringType(), True),
 StructField('HR16S1', StringType(), True),
 StructField('HR16S2', StringType(), True),
 StructField('HR17Val', LongType(), True),
 StructField('HR17MF', StringType(), True),
 StructField('HR17QF', StringType(), True),
 StructField('HR17S1', StringType(), True),
 StructField('HR17S2', StringType(), True),
 StructField('HR18Val', LongType(), True),
 StructField('HR18MF', StringType(), True),
 StructField('HR18QF', StringType(), True),
 StructField('HR18S1', StringType(), True),
 StructField('HR18S2', StringType(), True),
 StructField('HR19Val', LongType(), True),
 StructField('HR19MF', StringType(), True),
 StructField('HR19QF', StringType(), True),
 StructField('HR19S1', StringType(), True),
 StructField('HR19S2', StringType(), True),
 StructField('HR20Val', LongType(), True),
 StructField('HR20MF', StringType(), True),
 StructField('HR20QF', StringType(), True),
 StructField('HR20S1', StringType(), True),
 StructField('HR20S2', StringType(), True),
 StructField('HR21Val', LongType(), True),
 StructField('HR21MF', StringType(), True),
 StructField('HR21QF', StringType(), True),
 StructField('HR21S1', StringType(), True),
 StructField('HR21S2', StringType(), True),
 StructField('HR22Val', LongType(), True),
 StructField('HR22MF', StringType(), True),
 StructField('HR22QF', StringType(), True),
 StructField('HR22S1', StringType(), True),
 StructField('HR22S2', StringType(), True),
 StructField('HR23Val', LongType(), True),
 StructField('HR23MF', StringType(), True),
 StructField('HR23QF', StringType(), True),
 StructField('HR23S1', StringType(), True),
 StructField('HR23S2', StringType(), True),
 StructField('DlySum', LongType(), True),
 StructField('DlySumMF', StringType(), True),
 StructField('DlySumQF', StringType(), True),
 StructField('DlySumS1', StringType(), True),
 StructField('DlySumS2', StringType(), True)])

In [10]:
df = spark.read.schema(station_schema).parquet('s3a://weather-data-kpde/raw/*')

In [11]:
df.count()

23259498

In [12]:
df.rdd.getNumPartitions()

45

In [10]:
# df.coalesce(1).write.mode('overwrite').parquet('s3a://weather-data-kpde/out/')

23/01/30 19:42:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [13]:
# function to generate hourly columns
def generate_cols_as_array(col_name):
    return array([col(f'HR{i:02}{col_name}') for i in range(24)]).alias(col_name)

In [14]:
# unstack hourly columns
col_types = ['Val', 'MF', 'QF', 'S1', 'S2']

df_t = (df.select('STATION', 'DATE', arrays_zip(*[generate_cols_as_array(c) for c in col_types]
                                          ).alias('zip'))  # generate zipped arrays for each hour
 .select('*', posexplode('zip').alias('hr', 'exp'))  # explode zipped arrays
 .select('*', *[col('exp')[c].alias(c) for c in col_types],  # extract array
        )
 .select('STATION', 'DATE', 'hr', *col_types)
)

In [15]:
# replace -9999 values and ' ' strings with null
df_t = (df_t.replace({-9999: None}, subset=['Val'])
     .replace({' ': None}, subset=['MF', 'QF', 'S1', 'S2'])
)

In [18]:
df_t = df_t.withColumn('year', year('DATE'))

In [20]:
df_t.rdd.getNumPartitions()

45

In [21]:
df_t.write.mode('overwrite').parquet('s3a://weather-data-kpde/out/')

In [19]:
df_t.show(5)

+-----------+----------+---+---+---+----+---+----+----+
|    STATION|      DATE| hr|Val| MF|  QF| S1|  S2|year|
+-----------+----------+---+---+---+----+---+----+----+
|USC00355969|1948-07-01|  0|  0|  Z|null|  4|null|1948|
|USC00355969|1948-07-01|  1|  0|  Z|null|  4|null|1948|
|USC00355969|1948-07-01|  2|  0|  Z|null|  4|null|1948|
|USC00355969|1948-07-01|  3|  0|  Z|null|  4|null|1948|
|USC00355969|1948-07-01|  4|  0|  Z|null|  4|null|1948|
+-----------+----------+---+---+---+----+---+----+----+
only showing top 5 rows



In [17]:
# repartitioning is very expensive
# df_t.write.mode('overwrite').partitionBy('year').parquet('s3a://weather-data-kpde/out/')

23/02/01 16:12:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


ERROR:root:KeyboardInterrupt while sending command.                (8 + 4) / 45]
Traceback (most recent call last):
  File "/home/kishan/spark/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/kishan/spark/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [17]:
df.count()

27799

In [11]:
spark.stop()